In [4]:
import tkinter as tk

X = "X"
O = "O"
EMPTY = None

def initial_state():
    """
    Returns starting state of the board.
    """
    return [[EMPTY, EMPTY, EMPTY],
            [EMPTY, EMPTY, EMPTY],
            [EMPTY, EMPTY, EMPTY]]

def player(board):
    """
    Returns player who has the next turn on a board.
    """
    x_count = sum(row.count(X) for row in board)
    o_count = sum(row.count(O) for row in board)
    return X if x_count == o_count else O

def actions(board):
    """
    Returns set of all possible actions (i, j) available on the board.
    """
    possible_actions = set()
    for i in range(3):
        for j in range(3):
            if board[i][j] == EMPTY:
                possible_actions.add((i, j))
    return possible_actions

def result(board, action):
    """
    Returns the board that results from making move (i, j) on the board.
    """
    i, j = action
    new_board = [row[:] for row in board]
    new_board[i][j] = player(board)
    return new_board

def winner(board):
    """
    Returns the winner of the game, if there is one.
    """
    for row in board:
        if all(cell == X for cell in row):
            return X
        elif all(cell == O for cell in row):
            return O

    for j in range(3):
        if all(board[i][j] == X for i in range(3)):
            return X
        elif all(board[i][j] == O for i in range(3)):
            return O

    if all(board[i][i] == X for i in range(3)):
        return X
    elif all(board[i][i] == O for i in range(3)):
        return O

    if all(board[i][2 - i] == X for i in range(3)):
        return X
    elif all(board[i][2 - i] == O for i in range(3)):
        return O

    return None

def terminal(board):
    """
    Returns True if game is over, False otherwise.
    """
    return winner(board) is not None or all(all(cell is not None for cell in row) for row in board)

def utility(board):
    """
    Returns 1 if X has won the game, -1 if O has won, 0 otherwise.
    """
    result = winner(board)
    if result == X:
        return 1
    elif result == O:
        return -1
    else:
        return 0

def minimax(board):
    """
    Returns the optimal action for the current player on the board.
    """
    def max_value(board):
        if terminal(board):
            return utility(board), None
        v = float("-inf")
        best_action = None
        for action in actions(board):
            min_val, _ = min_value(result(board, action))
            if min_val > v:
                v = min_val
                best_action = action
        return v, best_action

    def min_value(board):
        if terminal(board):
            return utility(board), None
        v = float("inf")
        best_action = None
        for action in actions(board):
            max_val, _ = max_value(result(board, action))
            if max_val < v:
                v = max_val
                best_action = action
        return v, best_action

    if player(board) == X:
        return max_value(board)[1]
    else:
        return min_value(board)[1]

# GUI functions

def update_board(button, row, col):
    """
    Updates the board when a button is clicked.
    """
    global board

    if board[row][col] is not None or terminal(board):
        return

    button.config(text=player(board))
    board = result(board, (row, col))

    if terminal(board):
        game_winner = winner(board)
        if game_winner:
            status_label.config(text=f"Game over. The winner is {game_winner}")
        else:
            status_label.config(text="Game over. It's a tie.")
    elif player(board) == O:
        ai_move()

def create_button(row, col):
    """
    Creates a button for the GUI and registers the click event.
    """
    button = tk.Button(root, text=" ", font=("Arial", 24), width=4, height=2)
    button.grid(row=row, column=col)
    button.config(command=lambda: update_board(button, row, col))
    buttons[row][col] = button  # Update buttons list

def create_board_buttons():
    """
    Creates buttons for each cell in the board.
    """
    for row in range(3):
        for col in range(3):
            create_button(row, col)

def ai_move():
    """
    Makes a move for the AI player.
    """
    global board

    if terminal(board):
        return

    action = minimax(board)
    row, col = action
    button = buttons[row][col]
    update_board(button, row, col)

# Initialize the board
board = initial_state()

# Create the GUI window
root = tk.Tk()
root.title("Tic-Tac-Toe")

# Create buttons for the board
buttons = [[None, None, None],
           [None, None, None],
           [None, None, None]]
create_board_buttons()

# Create a status label
status_label = tk.Label(root, text="Player X's turn", font=("Arial", 16))
status_label.grid(row=3, columnspan=3)

# Start the GUI event loop
root.mainloop()